# Huaibin Ge

In [1]:
import pandas as pd
import numpy as np
import math
import copy
import csv

# Part I

## A

## 1. Scratch Multinomial classifier with laplace smothing

In [2]:
train_data = pd.read_csv('./hw1_trainingset.csv')
test_data = pd.read_csv('./hw1_testset.csv')

In [3]:
# validation_data = pd.read_csv('hw1_validationset.csv')
# vfeatures, vlabels = validation_data.drop(['Label'],axis=1), np.array(validation_data['Label'], header = False)

In [4]:
train_DV = train_data['Label']
train_IVs = train_data.drop(columns = ['Label'])

In [5]:
train_data.isnull().values.any()

False

In [6]:
test_data.isnull().values.any()

False

In [7]:
# test_data.iloc[0,1]

In [8]:
# unique, counts = np.unique(feature[0], return_counts=True)
# unique[0]
# counts[0]

In [9]:
def train_NBmultinomial(train_data):
    Class0 = train_data.where(train_data.iloc[:,-1]==0).dropna()
    Class1 = train_data.where(train_data.iloc[:,-1]==1).dropna()
    train_data = np.array(train_data, dtype = float)
    Class0 = np.array(Class0)
    Class1 = np.array(Class1)
    IVs = [data[:-1] for data in train_data]
    DVs = [data[-1] for data in train_data]
    nrow = len(DVs)
    nrow0 = len(Class0)
    nrow1 = len(Class1)
    P0 =DVs.count(0)/nrow
    P1 =DVs.count(1)/nrow
#     feature = np.zeros(shape = (IV_train.shape[1], IV_train.shape[0]), dtype = float)
#     for i in np.arange(IV_train.shape[1]):
#         for j in np.arange(IV_train.shape[0]):
#             feature[i,j] = IV_train.iloc[j,i]
    feature_class0 = []
    feature_class1 = []
    for i in range(len(IVs[0])):
        feature_temp0 = [data[i] for data in Class0]
        feature_temp1 = [data[i] for data in Class1]
        feature_class0.append(feature_temp0)
        feature_class1.append(feature_temp1)
    def classifier(test_data):
        test_data = np.array(test_data, dtype = float)
        predict = []
        for data in test_data:

            sum_pro0 = 0
            sum_pro1 = 0
            for i in range(len(test_data[0])):
                
                pro0 = feature_class0[i].count(data[i])/nrow0
                pro1 = feature_class1[i].count(data[i])/nrow1
                # k==6?
                if pro0 == 0:
                    pro0 = 1/(nrow0+len(test_data[0]))
                if pro1 == 0:
                    pro1 = 1/(nrow0+len(test_data[0]))
                multi_prob0 = math.log(pro0)
                sum_pro0 = sum_pro0 + multi_prob0
                
                prob1 = math.log(pro1)
                sum_pro1 = sum_pro1 + prob1
            sum_prob0_total = sum_pro0 + math.log(P0) 
            sum_prob1_total = sum_pro1 + math.log(P1)
            if sum_prob0_total>sum_prob1_total:
                predict.append(0)
            else:
                predict.append(1)
        return predict
    return classifier
            
            

def predict(clf, test_row):
    result = clf(test_row)
    return result


In [10]:
clf_multinomial = train_NBmultinomial(train_data)
prediction_multinomial = predict(clf_multinomial, test_data)

In [11]:
len(prediction_multinomial)

2400

## 2. Scratch Gaussian Bayes classifier

In [12]:
def train_NBGaussian(train_data):
    Class0 = train_data.where(train_data.iloc[:,-1]==0).dropna()
    Class1 = train_data.where(train_data.iloc[:,-1]==1).dropna()
    train_data = np.array(train_data, dtype = float)
    Class0 = np.array(Class0)
    Class1 = np.array(Class1)
    IVs = [data[:-1] for data in train_data]
    DVs = [data[-1] for data in train_data]
    nrow = len(DVs)
    nrow0 = len(Class0)
    nrow1 = len(Class1)
    P0 =DVs.count(0)/nrow
    P1 =DVs.count(1)/nrow
    feature_class0 = []
    feature_class1 = []
    mean0 = []
    var0  = []
    mean1 = []
    var1 = []
    for i in range(len(IVs[0])):
        feature_temp0 = [data[i] for data in Class0]
        feature_temp1 = [data[i] for data in Class1]
        feature_class0.append(feature_temp0)
        feature_class1.append(feature_temp1)
    for j in range(len(IVs[0])):
        temp_mean0 = np.mean(feature_class0[j])
        temp_mean1 = np.mean(feature_class1[j])
        temp_var0 = np.var(feature_class0[j])
        temp_var1 = np.var(feature_class1[j])
        
        mean0.append(temp_mean0)
        mean1.append(temp_mean1)
        var0.append(temp_var0)
        var1.append(temp_var1)
    #print(var0, var1)
    #print(mean0, mean1, var0, var1)    
    def classifier(test_data):
        #print(test_data.isnull().values.any())
        test_data = np.array(test_data, dtype = float)
        predict = []
        for data in test_data:

            sum_pro0 = 0
            sum_pro1 = 0
            for i in range(len(test_data[0])):
                #print(data[i])
                #print(var0[i], var1[i])
                pro0 = np.exp(((data[i]-mean0[i])**2)/(2*var0[i]))/np.sqrt(2*math.pi*var0[i])
                pro1 = np.exp(((data[i]-mean1[i])**2)/(2*var1[i]))/np.sqrt(2*math.pi*var1[i]) 
                
                #print(pro0)
                multi_prob0 = np.log(pro0)
                sum_pro0 = sum_pro0 + multi_prob0
                
                multi_prob1 = np.log(pro1)
                sum_pro1 = sum_pro1 + multi_prob1
            sum_prob0_total = sum_pro0 + np.log(P0) 
            sum_prob1_total = sum_pro1 + np.log(P1)
            
            if sum_prob0_total>sum_prob1_total:
                predict.append(0)
            elif sum_prob0_total<=sum_prob1_total:
                
                predict.append(1)
        return predict
    return classifier

In [13]:
clf_Gaussian = train_NBGaussian(train_data)
print(clf_Gaussian)
prediction_Gaussian = predict(clf_Gaussian, test_data)

<function train_NBGaussian.<locals>.classifier at 0x11fba5bf8>


In [14]:
print(len(prediction_Gaussian))

2400


## B. Scratch K-fold cross-validation

In [15]:
def cross_validation(model, train_data, k):
    train_data = np.array(train_data)
    split_data = np.split(train_data, k)
    f1_score = []
    
    for i in range(k):
        true_positive=0
        false_positive=0
        false_negative=0
        true_negative=0
        
        train = np.delete(train_data, range(i*560, (i+1)*560), axis=0)
        train1 = pd.DataFrame(train)
        clf = model(train1)
        test_data = [data[:-1] for data in split_data[i]]
        test_label = [data[-1] for data in split_data[i]]
        prediction = predict(clf, test_data)
        for j in range(len(prediction)):
            if prediction[j] == 1 and test_label[j] == 1:
                true_positive +=1
            elif prediction[j] == 1 and test_label[j] == 0:
                false_positive +=1
            elif prediction[j] == 0 and test_label[j] == 1:
                false_negative +=1
            elif prediction[j] == 0 and test_label[j] == 0:
                true_negative +=1
    #print(true_positive)
        f1 = 2*true_positive/(2*true_positive+false_positive+false_negative)
        f1_score.append(f1)
    return np.mean(f1_score)
            
            
            

## C. Compare performance

In [16]:
f1_NBmultinomial = cross_validation(train_NBmultinomial, train_data, 5)
f1_NBmultinomial

0.8720743484935326

In [17]:
f1_Gaussian = cross_validation(train_NBGaussian, train_data, 5)
f1_Gaussian

0.0008658008658008658

Multinomial Naive Bayes performs better.

## D. Compare the performance of above scratched function with scikit learn

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

In [19]:
clf_skGaussian = GaussianNB()
clf_skGaussian.fit(train_IVs, train_DV)
prediction_skGaussian = clf_skGaussian.predict(train_IVs)
f1_score(train_DV, prediction_skGaussian)

0.31512224569876246

In [20]:
train_IVs1 = train_IVs
train_IVs1['Feature_3'] += -np.min(train_IVs)[2]
clf_MNB = MultinomialNB()
clf_MNB.fit(train_IVs1, train_DV)
prediction_skMNB = clf_MNB.predict(train_IVs1)
f1_score(train_DV, prediction_skGaussian)

0.31512224569876246

## Test data

In [21]:
clf_multinomial = train_NBmultinomial(train_data)
prediction_multinomial = predict(clf_multinomial, test_data)

In [22]:
test_features = test_data

In [23]:
test_features["Label"] = prediction_multinomial
test_features.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Label
0,13876.40,140984.0,-5240.00,33,315,5844,0
1,9958.06,27101.6,-3330.73,27,174,1415,0
2,8716.96,10783.7,-3113.44,34,156,654,0
3,15707.20,116106.0,-5050.00,54,271,4426,0
4,5189.89,1928.0,-1113.00,14,68,149,1


In [24]:
test_features.to_csv("test_features.csv",index=False)

# Part II Scratch Association Rule

In [25]:
from itertools import combinations

In [26]:
basket_data = [[1,1,0,1,0,0], [1,1,0,0,0,1], [0,1,1,0,0,1], [0,0,1,0,1,0], [1,0,1,0,0,1],
              [1,1,0,0,1,0],[0,1,0,0,1,1]]

In [27]:
def APriori(basket_data, support, confidence, lift):
    
    def join(List, j):
        List_new = []
        if j == 1:
            for item in combinations(List, 2):
                List_new.append(item)   
            
        else:
            for item in combinations(List, 2):
                new = list(item[0])+list(item[1])
                new = set(new)
                if len(new) == j+1:
                    List_new.append(tuple(new))
            List_new = set(List_new)
        
        return list(List_new)
 
    
    
    def support_rule(basket_key, allcandidate, support):
    # subset
        Can = []
        Can_final = []
        for key in basket_key:
            for candidate in allcandidate:
                if set(candidate).issubset(set(key)):
                    Can.append(candidate)
        
    # support(tuple of list)
        for can in set(Can):
            if Can.count(can)/(len(basket_data))>support:
                Can_final.append(can)
        Can_final = set(Can_final)
        
        return Can_final
                  
    
    def prune(Lj, Lj_1, j):
        
        Ljj = copy.deepcopy(Lj)
        
        if j == 1:
            return Lj
        else:
            for items in Lj:
                for com in combinations(items, j):
                    if com not in Lj_1:
                        if items in Ljj:
                            Ljj.remove(items)
        return Ljj
    
    
    
    # find L1
    count = []
    key_L1 = []
    basket_key = []
    L = []
    L_final = []

    for i in range(len(basket_data[0])):
        item = [data[i] for data in basket_data]
        count.append(item.count(1))
    
    # basketball_key
    for data in basket_data:
        standard_key = list(range(1, len(basket_data[0])+1))
        for i in range(len(basket_data[0])):
            if data[i] == 0:
                standard_key[i] = 0
        basket_key.append(standard_key)
        
    for j in range(len(basket_data[0])):
        if count[j]/(len(basket_data))>support:
            key_L1.append(j+1)
    
    L.append(key_L1)
    l = 0
    
    while L[l]:
        C = join(L[l], l+1)
        #print(C)
        temp_L = support_rule(basket_key, C, support)
        temp_L = prune(temp_L, L[l], l+1)
        l = l+1
        L.append(temp_L)
    for i in range(1,len(L)):
        for j in range(len(L[i])):
            lll = list(L[i])[j]
#             kkk = set(lll)
            L_final.append(lll)
    #print(L_final)
    # Confidence
    
    def confidence_value(data, b, a):
        counta = 0
        countb = 0
        con_key = []
        for key in data:
            if b.issubset(set(key)):
                con_key.append(key)
                countb += 1
        for key in con_key:
            if a.issubset(set(key)):
                counta += 1
        confidence = counta/countb
        #print(confidence)
        return confidence
    
    def lift_value(data, b, a):
        counta = 0
        countb = 0
        ca = 0
        pa = 0
        con_key = []
        for key in data:
            if b.issubset(set(key)):
                con_key.append(key)
                countb += 1
        for key in con_key:
            if a.issubset(set(key)):
                counta += 1
                
        for key in data:
            if a.issubset(set(key)):
                ca += 1
            
        confidence = counta/countb
        pa = ca/len(data)
        lift = confidence/pa
        #print(confidence, pb)
        return lift
    
    def subset(item):
        k = len(item)
        sub = []
        for i in range(k-1):
            for it in combinations(item, i+1):
                sub.append(set(it))
        return sub
    
    def conversion(a,b):
        model = ["A", "B", "C", "D", "E", "F"]
        output_final = []
        outputa = []
        outputb = []
        for index in a:
            output = model[index-1]
            outputa.append(output)
        for index in b:
            output = model[index-1]
            outputb.append(output)
        output_final.append('''{} => {}'''.format(set(outputa), set(outputb)))
        
        return output_final
        #return print('''{} => {}'''.format(set(outputa), set(outputb)))
    
    
    
    sol = []
    for item in L_final:
        item = set(item)
        if len(item)>=2:
            for a in subset(item):
                #print(subset(item))
                k = item - a
                #print(k)
                if confidence_value(basket_key, item-a, a)  >= confidence and lift_value(basket_key, item-a, a)>lift:
                    #sol.append([item-a, a])
                    aaa = conversion(item-a, a)
                    sol = sol+aaa
    return sol
    
    

In [28]:
results = {}

In [29]:
rul1 = APriori(basket_data, 0.1, 0.3, 1)
print(rul1)
results["support=0.1,confidence=0.3,lift=1"] = list(rul1)

["{'B'} => {'A'}", "{'A'} => {'B'}", "{'F'} => {'B'}", "{'B'} => {'F'}", "{'D'} => {'A'}", "{'F'} => {'C'}", "{'C'} => {'F'}", "{'D'} => {'B'}", "{'C', 'B'} => {'F'}", "{'A', 'F'} => {'C'}", "{'C', 'A'} => {'F'}", "{'C'} => {'A', 'F'}", "{'B', 'D'} => {'A'}", "{'A', 'D'} => {'B'}", "{'A', 'B'} => {'D'}", "{'D'} => {'A', 'B'}", "{'F', 'E'} => {'B'}", "{'A', 'E'} => {'B'}"]


In [30]:
rul2 = APriori(basket_data, 0.2, 0.65, 0)
print(rul2)
results["support=0.2,confidence=0.65,lift=0"] = list(rul2)

["{'A'} => {'B'}", "{'F'} => {'B'}", "{'C'} => {'F'}", "{'E'} => {'B'}"]


In [31]:
rul3 = APriori(basket_data, 0.2, 0.65, 1.1)
print(rul3)
results["support=0.2,confidence=0.65,lift=1.1"] = list(rul3)

["{'C'} => {'F'}"]


In [32]:
rul4 = APriori(basket_data, 0.2, 0.65, 2)
print(rul4)
results["support=0.2,confidence=0.65,lift=2"] = list(rul4)

[]


In [33]:
rul5 = APriori(basket_data, 0.2, 0.5, 1.1)
print(rul5)
results["support=0.2,confidence=0.5,lift=1.1"] = list(rul5)

["{'F'} => {'C'}", "{'C'} => {'F'}"]


In [34]:
rul6 = APriori(basket_data, 0.3, 0.7, 1.1)
print(rul6)
results["support=0.3,confidence=0.7,lift=1.1"] = list(rul6)

[]


In [35]:
Result = pd.DataFrame(dict([(i, pd.Series(j)) for i, j in results.items()]))

In [36]:
Result

,"support=0.1,confidence=0.3,lift=1","support=0.2,confidence=0.65,lift=0","support=0.2,confidence=0.65,lift=1.1","support=0.2,confidence=0.65,lift=2","support=0.2,confidence=0.5,lift=1.1","support=0.3,confidence=0.7,lift=1.1"
0,{'B'} => {'A'},{'A'} => {'B'},{'C'} => {'F'},NaN,{'F'} => {'C'},NaN
1,{'A'} => {'B'},{'F'} => {'B'},NaN,NaN,{'C'} => {'F'},NaN
2,{'F'} => {'B'},{'C'} => {'F'},NaN,NaN,NaN,NaN
3,{'B'} => {'F'},{'E'} => {'B'},NaN,NaN,NaN,NaN
4,{'D'} => {'A'},NaN,NaN,NaN,NaN,NaN
5,{'F'} => {'C'},NaN,NaN,NaN,NaN,NaN
6,{'C'} => {'F'},NaN,NaN,NaN,NaN,NaN
7,{'D'} => {'B'},NaN,NaN,NaN,NaN,NaN
8,"{'C', 'B'} => {'F'}",NaN,NaN,NaN,NaN,NaN
9,"{'A', 'F'} => {'C'}",NaN,NaN,NaN,NaN,NaN


In [37]:
Result.to_csv('Result.csv', index = False)